In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import asyncio

from ezib_async.ezib import ezIBAsync
from ezib_async import util

In [ ]:
util.logToConsole('DEBUG')
logging.getLogger('ib_async').setLevel('CRITICAL')

In [ ]:
# initialize ezIBAsync
ezib = ezIBAsync()

await ezib.connectAsync(ibclient=100, ibhost="localhost", ibport=4001, account='U9860850')
# await ezib.connectAsync(ibport=4002, ibclient=100)

In [ ]:
from ib_async import Forex, Contract
# contract = Forex('EURUSD')
contract = Forex(pair='EURUSD')
await ezib.ib.qualifyContractsAsync(contract)

In [ ]:
contract

In [ ]:
ezib.requestMarketDepth(contract)

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5),
        columns='bidSize bidPrice askPrice askSize'.split())

def onTickerUpdate(ticker):
    # print(ticker.domAsks)
    orderbook = ticker
    orderbook.symbol = 'symbol'
    print(orderbook.symbol)
    bids = orderbook.domBids
    for i in range(5):
        df.iloc[i, 0] = bids[i].size if i < len(bids) else 0
        df.iloc[i, 1] = bids[i].price if i < len(bids) else 0
    asks = orderbook.domAsks
    for i in range(5):
        df.iloc[i, 2] = asks[i].price if i < len(asks) else 0
        df.iloc[i, 3] = asks[i].size if i < len(asks) else 0
    clear_output(wait=True)
    display(df)

# ib.pendingTickersEvent += onTickerUpdate
ezib.updateMarketDepthEvent +=  onTickerUpdate
await asyncio.sleep(15)
# ib.pendingTickersEvent -= onTickerUpdate
ezib.updateMarketDepthEvent -= onTickerUpdate

In [ ]:
# create some market contracts using dedicated methods
contracts = await asyncio.gather(
    # market contracts
    ezib.createStockContract("NVDA"),
    ezib.createStockContract('PLTR'),
    ezib.createStockContract('NFLX'),
    ezib.createStockContract('META'),
    ezib.createStockContract('TSLA'),
    
    # option contracts
    ezib.createOptionContract("AAPL", expiry="20250530", strike=200, otype="P"),
    ezib.createOptionContract("AAPL", expiry="20250530", strike=200, otype="C")
    
    # futures contracts
    # ezib.createFuturesContract("ES", expiry="202506", exchange="CME"),
    # ezib.createContract(("CL", "FUT", "NYMEX", "USD", "202506", 0.0, "")),
    
    # forex contracts
    # ezib.createForexContract("EUR", currency="USD")
)

In [ ]:
util.df(ezib._contract_details)

In [ ]:
# request market data for all created contracts
await ezib.requestMarketData()

In [ ]:
util.df(ezib.ib.orders())

In [ ]:
import ib_async


[attr for attr in dir(ib_async.Ticker) if not attr.startswith('_')]

In [ ]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[ezib.contractString(c) for c in ezib.contracts if c.secType not in ("OPT", "FOP", "FUT")],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'last', 'prevLast', 'lastSize', 'prevLastSize','volume'])

def onMarketTickerUpdate(tickers):
    for t in tickers:
        df.loc[ezib.contractString(t.contract)] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.last, t.prevLast, t.lastSize, t.prevLastSize, t.volume)
        clear_output(wait=True) 
    display(df)
    # print(t)
    
ezib.pendingMarketTickersEvent += onMarketTickerUpdate
# await asyncio.sleep(30)
# ezib.pendingMarketTickersEvent -= onMarketTickerUpdate

In [ ]:
ezib.pendingMarketTickersEvent -= onMarketTickerUpdate

In [ ]:
util.df(ezib.portfolios)

In [ ]:
util.df(ezib.portfolio)

In [ ]:
ezib.isConnected

In [ ]:
# cancel market data request & disconnect
ezib.cancelMarketData()


In [ ]:
ezib.disconnect()